# V2

## Libraries Used

In [1]:
import numpy as np
from scipy.optimize import minimize_scalar, minimize 
import time
import matplotlib.pyplot as plt
import pandas as pd

Matplotlib is building the font cache; this may take a moment.


---

## Exact Line Search
- Computes the optimal step size along the gradient at each iteration 

In [2]:
# finding optimal alpha (step-size)
# g: function g(x), x: current point, direction: gradient at point x

# def exact_line_search(g, x, direction):
def exact_line_search(g, x, direction, A, b):
    # This is how we can write a function in one line
    # phi = lambda alpha: g(x + alpha * direction)
    
    def phi(alpha):
        return g(x - alpha*direction, A, b)
    
    # solves local minimization of scalar function of one variable
    # Runs our phi function and tests various alpha values (input) 
    # until it finds one that minimizes the function
    res = minimize_scalar(phi) # returns an OptimizeResult object

    return res.x # The value of alpha that minimizes phi

---

## Proximal Operators of L1 and L2 Euclidean Norms

In [3]:
# implements l1 norm of lambda * ||x||
def prox_l1_norm(x, alpha_lamb):
    # prox operator = max(|x| - λα, 0) * sign(x)
    return np.maximum(np.abs(x) - alpha_lamb, 0) * np.sign(x)

# implements l2 norm of lambda * ||x||_2
def prox_l2_norm(x, lamb):

    # prox operator = (1- λ/(max(||x||, λ))) * x
    
    normed = np.linalg.norm(x, 2)
    den = np.maximum(normed, lamb)
    return (1 - lamb/den) * x

---

## Subgradient of $\|Ax - b\|_1$

- The subgradient is given by: $A^T \cdot s$,  
  where $A^T$ is the transpose of $A$, and $s$ is the sign vector of $(Ax - b)$.

In [4]:
def sub_grad_g(x, A, b, tol=1e-6):

    # say r = Ax-b
    r = A @ x - b             # shape (m,)
    # r = (A @ x - b).flatten()
    
    # sign matrix intialization (for indexing)
    s = np.zeros_like(r)      # shape (m,)

    # for every row or entry in the vector
    for i in range(len(r)):
        # if positive val, sign = 1
        if r[i] > tol:
            s[i] = 1.0
        # if negative val, sign = -1
        elif r[i] < -tol:
            s[i] = -1.0
        else:
            # r[i] is close to 0, pick any subgradient in [-1, 1].
            # We'll pick 0 for simplicity.
            s[i] = 0.0
    
    # Now the subgradient w.r.t x is A^T @ s
    g_sub = A.T @ s  # shape (n,)
    # g_sub = A.T @ np.sign(r)  
    return g_sub

---

## Functions: $g(x) = \|Ax - b\|_1$ and $h(x) = \|x\|_1$

- Where:
  - Matrix $A \in \mathbb{R}^{m \times n}$  
  - Vector $b \in \mathbb{R}^{m}$ 
  - Vector $x \in \mathbb{R}^{n}$
    - $n$ being the number of columns in $A$

In [5]:
# A * x results in a product matrix of mx1, the same dimensions of b,
# thus, you can now substract the two

def g(x, A, b):
    # @ is used for matrix multiplication
    return np.linalg.norm(A @ x - b, ord=1)

def h(x):
    # Convert x to a 1D array before computing the norm by using flatten()
    # NumPy treats a column vector differently from a 1D array. 
        # If x is a 2D array with shape (n, 1), np.linalg.norm(x, ord=1) will fail.
    return np.linalg.norm(x.flatten(), ord=1)

---

## Proximal Gradient Descent Method

In [6]:
# added A and b
# def proximal_descent(g, g_prime, h, h_prox, x0, A, b, lamb, iterations = 20000, alpha = 1e-6, tol = 1e-4):
def proximal_descent(g, g_prime, h, h_prox, x0, A, b, lamb, alpha = 1, iterations = 200, tol = 1e-4):
    """
    minimizes a non-differentiable function f(x) = g(x) + h(x)
    PARAMS
    g: function
        g(x), the differentiable part of f
    
    g_prime: function
        g'(x) aka the gradient of g
        returns the direction of steepest increase along g

    h: function
        h(x), the non-differentiable part of f
        
    h_prox: function
        h_prox(x, alpha) returns proximal operator of h at x using alpha as a distance weighting param
        h_prox gives a new x' which is a tradeoff of reducing h and staying close to x
        
    x0: vector
        initial starting point

    A: mxn matrix

    b: mx1 vector

    lamb: lambda value used in g + λ*h

    alpha: step size
        
    iterations: self explanitory
    
    tol: self explanitory
    
    RETURNS
    x* = argmin_x { f(x) } if x* is reachable in the given num iterations along with following relevant data
    """
    # initialize current guess at x0
    xk = x0

    func_values = [] # tracks the function values at each iteration
    x_differences = [] # tracks the norm of the step size
    func_differences = [] # tracks the difference of function values between xk (current) and xk_old
    gradient_norms = [] # tracks the norm of the gradient at each iteration
    h_norms = [] # tracks the norm of h at each iteration
    alphas = [] # tracks the computed optimal alpha at each iteration

    # for k iterations
    for k in range(1, iterations+1):
        # store old xk value
        xk_old = xk

        # compute gradient for differentiable part of f
        gk_gradient = g_prime(xk, A, b)
        
        # keep track of the norm of the gradient in every iteration
        gradient_norms.append(np.linalg.norm(gk_gradient))

        # find optimal step size along the gradient using an exact line search
        alpha_k = exact_line_search(g, xk, gk_gradient, A, b)
        
        # keep track of the optimal computed alpha in every iteration
        alphas.append(alpha_k)
        
        # take gradient step to reduce g(x)
        gradient_step = xk - alpha_k * gk_gradient  
        
        # find new x point 
        xk = h_prox(gradient_step, alpha_k*lamb)

        # compute the function value at the current xk value
        func_val_xk = g(xk, A, b) + lamb * h(xk)

        # keep track of each computed function value in each iteration
        func_values.append(func_val_xk)

        # compute the function value at the old xk value
        func_val_xold = g(xk_old, A, b) + lamb * h(xk_old)

        # Compute the difference of function values between xk (current) and xk_old
        func_differences.append(abs(func_val_xk - func_val_xold))
        
        # keep track of the norm of h evalauted at xk
            # Helps analyze if the function is decreasing in every iteration (minimizing)
        h_norms.append(np.linalg.norm(h(xk)))

        # keep track of the norm of the step size
        x_differences.append(np.linalg.norm(xk - xk_old, ord=2))

        # Check if the norm of the step size is under our accepted tolerance
        if np.linalg.norm(xk - xk_old, ord=2) < tol:
            # If so we say it converged
            print("Converged!")

            # return the following data
            return xk, k, func_values, x_differences, func_differences, gradient_norms, alphas, h_norms

    # Did not converge if the norm of the step size was not under our tolerance 
    print("Did not converge!")

    # return the following data
    return xk, k, func_values, x_differences, func_differences, gradient_norms, alphas, h_norms

---

## `SetUpProblem()`
##### Initializes the true $x$ vector, matrix $A$, and vector $b$.
- **Parameters:**
  - $m$: number of rows
  - $n$: number of columns
  - $lamb$: lambda
  - *case_num*: Test problem number
   
- **Returns**
  - Matrix $A \in \mathbb{R}^{m \times n}$
  - Vector $b \in \mathbb{R}^{m}$
  - True solution vector $x \in \mathbb{R}^{n}$  

In [7]:
def SetUpProblem(m, n, lamb, case_num):
    match case_num:
        case 1:
            # Assume the true solution of x to be a vector of all 1's
            x_true = np.ones((n, 1))

            # random mxn matrix
            A = np.random.rand(m, n) 

            # force vector be to be equal to Ax
                # Cancels out the first term g(x)
            b = A @ x_true 

            return A, b, x_true
        case 2:
            # Assume the true solution of x to be a vector of all 0's except let the first entry be 1
            x_true = np.zeros((n, 1))
            x_true[0,0] = 1

            # random mxn matrix
            A = np.random.rand(m, n) 

            # force vector be to be equal to Ax
                # Cancels out the first term g(x)
            b = A @ x_true 

            return A, b, x_true
        case _:
            return "Case number not found or not passed"

## `run_problem()`

- The function runs the **Proximal Gradient Method** using the following parameters passed in.
  - Evaluates the algorithm using an initial $x$ vector of all 0's
- **Returns**
  - **final_x**
    - Final $x$ vector that the Proximal Gradient Descent algorithm returned (regardless of convergence or not)
  - **itera**
    - Number of iterations it took to converge/not converge (max iterations)
  - **func_values**
    - List of function values computed at current x-vector at each iteration
  - **x_diffs**
    - List of the norm of the step size at each iteration
  - **func_diffs**
    - List of difference in function values evaluated at xk (current) and xk_old (x-vector from the previous iteration)
  - **grads**
    - List of the norm of the gradient at each iteration
  - **returned_alphas**
    - List of optimized alphas computed at each iteration
  - **h_norms**
    - List of the norm of h computed at the current x-vector (xk) at each iteration
  - **t_total**
    - Time spent inside the Proximal Gradient Descent algorithm
  - **init_func_val**
    - Initial function value evaluated at x_0
  - **final_func_val**
    - Final function value evaluated at final_x 

In [8]:
def run_problem(g, sub_grad_g, h, prox_oper, lamb, init_alpha, A, b):

    """
    g: g(x) function that has a computable sub-gradient
    
    sub_grad_g: The subgradient of g(x)
    
    h: The function h(x) that is not necessarily differentiable 
    
    prox_oper: Function computing the proximal operator of an L1 norm
    
    lamb: Lambda value
    
    init_alpha: Initial step size to use for the Proximal Gradient Descent Method
    
    A: mxn matrix

    b: mx1 matrix
        
    """
    
    # Set the initial vector x_0 to be all zeros
    x_0 = np.zeros((A.shape[0], 1)) # nx1 vector of all 0's

    # Compute function value at x_0
    init_func_val = g(x_0, A, b) + lamb*h(x_0)

    # Start the timer
    t0 = time.time() 

    # call the Proximal Gradient Descent Method
    final_x, itera, func_values, x_diffs, func_diffs, grads, returned_alphas, h_norms = proximal_descent(g, sub_grad_g, h, prox_oper, x_0, A, b, lamb, init_alpha)

    # End the timer
    tf = time.time() 

    # Compute the total elpased time
    t_total = tf-t0 

    # Compute final function value at final_x
    final_func_val = g(final_x, A, b) + lamb*h(final_x)

    return final_x, itera, func_values, x_diffs, func_diffs, grads, returned_alphas, h_norms, t_total, init_func_val, final_func_val    

## Helper function to build a dataframe

In [9]:
def build_data_frame(func_values, func_diffs, x_diffs, returned_alphas, grads, h_norms):
    """
    Parameters are the values returned from proximal_descent
    
    func_values: 
        List of function values computed at current x-vector at each iteration
    
    func_diffs: 
        List of difference in function values evaluated at xk (current) and 
        xk_old (x-vector from the previous iteration)
    
    x_diffs: 
        List of the norm of the step size at each iteration
    
    returned_alphas: 
        List of optimized alphas computed at each iteration
    
    grads: 
        List of the norm of the gradient at each iteration
    
    h_norms: 
        List of the norm of h computed at the current x-vector (xk) at each iteration
    
    """
    
    # object to help create a pandas dataframe (table)
    data = {} 
    
    # data = {key: column_name, val: values to display}
    data['Function Value'] = func_values 
    data['Func diff'] = func_diffs
    data['||Step Size||'] = x_diffs
    data['alphas'] = returned_alphas
    data['||gradient||'] = grads
    data['h_norms'] = h_norms

    # create the table using the created object
    df = pd.DataFrame(data) 

    return df

## Helper function to print out:
- Matrix dimensions of $A$
- Initial function value: $f(x_0)$
- Final function value: $f(x_{final})$
  - **Note**: $x_{final}$ may not be the true $x$ that minimizes $\|Ax-b\|_1 + 2\|x\|_1$
    - e.g. non-convergence still returns some $x_{final}$
- Elapsed time spent in the **Proximal Gradient Descent Method**
- Absolute error between $\| x_{true} \|$ and $\| x_{final} \|$

In [10]:
def print_proximal_gd_metrics(m, n, init_func_val, final_func_val, time_elasped, final_x, x_true):
    """

    m,n: Matrix Dimensions of A

    init_func_val: Initial function value evaluated at x_0

    final_func_val: 
        Final function value evaluated at the last last x-vector that 
        the Proximal Gradient Descent algorithm returned

    time_elapsed: Time spent inside the Proximal Gradient Descent algorithm

    final_x: the last last x-vector that the Proximal Gradient Descent algorithm returned

    x_true: x-vector that we chose to which we know the expected solution of f(x_true)
    
    """
    
    print(f"Matrix Dimensions of A: {m}x{n}")
    print("--------------"*4)
    
    print(f"Initial Function Value f(x_0): {init_func_val}")

    print(f'Final Function Value f(x_final): {final_func_val}')

    print(f"Elapsed Time taken: {t_total}s")

    print(f"abs( ||x*|| - ||x|| ): {abs(np.linalg.norm(x_true) - np.linalg.norm(final_x))}")
    print()

---

## Test Problem 1

- $min \{ \|Ax-b\|_1 + \lambda \|x\|_1 \}$
  - $A$ be an $m \times n$ matrix
  - $x$ be an $n \times 1$ vector
  - $b$ be an $m \times 1$ vector  
  - $\lambda = 2$

- Assume the true solution of $x$ to be a vector of all 1's
    - Then, we expect $f_{min} = \lambda \|x\|_1 = 2n$, if $Ax=b$

In [11]:
# define a list of dictionaries that represent all the dimensions 
# that you want to test the algorithm on
dimensions = [{'m':5, 'n':5}, {'m':10, 'n':10}, {'m':20, 'n':20}, {'m':40, 'n':40}]

test_problem_num = 1

# Lambda value to use
lamb = 2

# Initial step size (alpha) to use
init_alpha = 1

# Intitialize matrix A, and vector's b and x_true based on the case number
for d in dimensions:
    A, b, x_true = SetUpProblem(d['m'], d['n'], lamb, test_problem_num)
    final_x, itera, func_values, x_diffs, func_diffs, grads, returned_alphas, h_norms, t_total, init_func_val, final_func_val = run_problem(g, sub_grad_g, h, prox_l1_norm, lamb, init_alpha, A, b)
    print_proximal_gd_metrics(d['m'], d['n'], init_func_val, final_func_val, t_total, final_x, x_true)
    df = build_data_frame(func_values, func_diffs, x_diffs, returned_alphas, grads, h_norms)
    print(df)

Did not converge!
Matrix Dimensions of A: 5x5
--------------------------------------------------------
Initial Function Value f(x_0): 14.267014958264063
Final Function Value f(x_final): 9.229497653058015
Elapsed Time taken: 0.7320001125335693s
abs( ||x*|| - ||x|| ): 0.14317380359242682

     Function Value  Func diff  ||Step Size||    alphas  ||gradient||  \
0         12.480825   1.786190       0.778179  0.339025      6.506664   
1         11.269700   1.211125       0.527644  0.229876      6.506664   
2         10.448498   0.821203       0.357769  0.155867      6.506664   
3          9.891682   0.556816       0.242585  0.105686      6.506664   
4          9.729664   0.162018       0.164484  0.071660      6.506664   
..              ...        ...            ...       ...           ...   
195        9.237296   0.019303       0.027497  0.039171      4.732540   
196        9.219614   0.017682       0.025189  0.035882      4.732540   
197        9.227808   0.008193       0.023074  0.032869

## Test Problem 2

- $min \{ \|Ax-b\|_1 + 2\|x\|_1 \}$
  - $A$ be an $m \times n$ matrix
  - $x$ be an $n \times 1$ vector 
  - $b$ be an $m \times 1$ vector  
  - $\lambda = 2$

- Assume the true solution of $x$ to be a vector of all 0's except let the first entry be 1
  - Then, we expect $f_{min} = \lambda \|x\|_1 = 2$, if $Ax=b$

In [12]:
# define a list of dictionaries that represent all the dimensions 
# that you want to test the algorithm on
dimensions = [{'m':5, 'n':5}, {'m':10, 'n':10}, {'m':20, 'n':20}, {'m':40, 'n':40}]

test_problem_num = 2

# Lambda value to use
lamb = 2

# Initial step size (alpha) to use
init_alpha = 1

# Intitialize matrix A, and vector's b and x_true based on the case number
for d in dimensions:
    A, b, x_true = SetUpProblem(d['m'], d['n'], lamb, test_problem_num)
    final_x, itera, func_values, x_diffs, func_diffs, grads, returned_alphas, h_norms, t_total, init_func_val, final_func_val = run_problem(g, sub_grad_g, h, prox_l1_norm, lamb, init_alpha, A, b)
    print_proximal_gd_metrics(d['m'], d['n'], init_func_val, final_func_val, t_total, final_x, x_true)
    df = build_data_frame(func_values, func_diffs, x_diffs, returned_alphas, grads, h_norms)
    print(df)

Did not converge!
Matrix Dimensions of A: 5x5
--------------------------------------------------------
Initial Function Value f(x_0): 3.1250919076888666
Final Function Value f(x_final): 2.0121095122553867
Elapsed Time taken: 0.4439997673034668s
abs( ||x*|| - ||x|| ): 0.024062566016034737

     Function Value  Func diff  ||Step Size||    alphas  ||gradient||  \
0          2.821959   0.303133       0.164532  0.089304      6.037234   
1          2.609666   0.212293       0.115227  0.062542      6.037234   
2          2.460992   0.148675       0.080697  0.043800      6.037234   
3          2.435111   0.025881       0.056514  0.030674      6.037234   
4          2.381204   0.053907       0.036736  0.037873      3.503554   
..              ...        ...            ...       ...           ...   
195        2.012811   0.000311       0.000665  0.001422      3.503554   
196        2.012519   0.000292       0.000670  0.001431      3.503554   
197        2.013103   0.000584       0.001500  0.0008

---